# NHL Official API Calls

# Imports

In [42]:
import requests
import pandas as pd
from functions import *
import pickle
import wikipedia
%run functions.py

# Making calls

## Getting Team Info

In [17]:
## I store this data in the functions file
# all_teams = make_bare_api_call("teams")
# team_ids = {item['name'] : item['id'] for item in all_teams['teams']}

In [26]:
# with open('data/team_id_dict.pickle', 'wb') as handle:
#     pickle.dump(team_ids, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Putting all player info into df

In [2]:
# all_players_df = get_all_player_ids_into_df()

In [3]:
# all_players_df2 = add_player_bithplace_to_df(all_players_df)

### Adding Team names to dataframe

In [15]:
# team_id_values = list(team_ids.values())
# team_id_index = [team_id_values.index(x) for x in all_players_df2.team_id]

In [19]:
# team_id_keys = list(team_ids.keys())
# all_players_df2['team_name'] = [team_id_keys[x] for x in team_id_index]

## Saving DataFrame 

In [27]:
# all_players_df2.to_csv("data/player_info_df.csv")

# Getting Location of each team

In [20]:
team_arenas = {item['name'] : item['venue']['name'] for item in all_teams['teams']}

In [24]:
# searching coords through wikipedia
wikipedia_titles = []
for x in team_arenas.values():
    try:
        title = wikipedia.page(x).title
    except wikipedia.exceptions.DisambiguationError:
        title = None
    except wikipedia.exceptions.PageError:
        title = "Google Search"
    wikipedia_titles.append(title)

In [29]:
coordinates_from_wikipedia = []
for title in wikipedia_titles:
    full_coords = None
    if title != 'Google Search':
        try:
            coords = wikipedia.WikipediaPage(title=title).coordinates
            str_lat = str(round(float(coords[0]), 4))
            str_lng = str(round(float(coords[1]), 4))
            full_coords = str_lat + ',' + str_lng
        except KeyError:
            full_coords = 'KeyError'
    coordinates_from_wikipedia.append(full_coords)

In [39]:
keyerror_indices = [idx for idx, x in enumerate(coordinates_from_wikipedia) if x == 'KeyError']
none_indices = [idx for idx, x in enumerate(wikipedia_titles) if x == 'Google Search']
all_missing_indices = keyerror_indices + none_indices
print(f"We are missing {len(all_missing_indices)} arena coordinates")

We are missing 4 arena coordinates


In [38]:
all_missing_indices

[1, 3, 5, 27]

In [48]:
## this helps me view the arenas that I'm missing
# for idx, x in enumerate(team_arenas.values()):
#     if idx in all_missing_indices:
#         print(x)

In [47]:
# this helps me correct the coordinate format I get from google
# coords = turn_coords_to_correct_format("44.9446° N, 93.1002° W")
# coords

In [49]:
# saving the coordinates
arena_coord_list = ['40.7233,-73.5907', "39.9012,-75.1720", '42.3662,-71.0621', '44.9446,-93.1002']

In [52]:
# re-adding the coordinates to the coordinate list
counter = 0
while counter < len(arena_coord_list):
    idx_to_replace = all_missing_indices[counter]
    coordinates_from_wikipedia[idx_to_replace] = arena_coord_list[counter]
    counter += 1

In [55]:
arena_df = pd.DataFrame.from_dict(team_arenas, orient='index')

In [57]:
arena_df['Coordinates'] = coordinates_from_wikipedia
arena_df.columns = ['Arena_name', 'Coordinates']

In [62]:
canadian_arenas = ["Bell Centre", "Canadian Tire Centre", "Scotiabank Arena", "Scotiabank Saddledome",
                  "Rogers Place", "Rogers Arena", "Bell MTS Place"]
canadian_teams = [True if x in canadian_arenas else False for x in arena_df['Arena_name']]
arena_df['Canadian_team'] = canadian_teams

## Saving Arena Info

In [64]:
arena_df.to_csv("data/arena_df.csv")